## 准备数据

In [70]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [71]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [72]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 128]), trainable=True, dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([128]), trainable=True, dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([128, 128]), trainable=True, dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([128]), trainable=True, dtype=tf.float32)
        self.W3 = tf.Variable(tf.random.normal([128, 10]), trainable=True, dtype=tf.float32)
        self.b3 = tf.Variable(tf.zeros([10]), trainable=True, dtype=tf.float32)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)
        logits = tf.matmul(h2, self.W3) + self.b3
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [73]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [74]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 814.01294 ; accuracy 0.109066665
epoch 1 : loss 512.6097 ; accuracy 0.13846667
epoch 2 : loss 376.79105 ; accuracy 0.17865
epoch 3 : loss 290.17673 ; accuracy 0.22251667
epoch 4 : loss 243.48723 ; accuracy 0.25998333
epoch 5 : loss 214.6767 ; accuracy 0.29545
epoch 6 : loss 194.4915 ; accuracy 0.32918334
epoch 7 : loss 178.74773 ; accuracy 0.36021668
epoch 8 : loss 165.99072 ; accuracy 0.3865
epoch 9 : loss 155.32288 ; accuracy 0.4099
epoch 10 : loss 146.3208 ; accuracy 0.43005
epoch 11 : loss 138.60358 ; accuracy 0.44908333
epoch 12 : loss 131.87793 ; accuracy 0.46543333
epoch 13 : loss 125.95625 ; accuracy 0.48108333
epoch 14 : loss 120.71206 ; accuracy 0.49428332
epoch 15 : loss 116.0088 ; accuracy 0.50666666
epoch 16 : loss 111.7618 ; accuracy 0.5172833
epoch 17 : loss 107.90667 ; accuracy 0.5280833
epoch 18 : loss 104.38815 ; accuracy 0.53735
epoch 19 : loss 101.15442 ; accuracy 0.5456667
epoch 20 : loss 98.17581 ; accuracy 0.5542
epoch 21 : loss 95.42558 ; accuracy